In [30]:
# ===============================
# Step 8: Deployment & Save Production Model (CLEAN OUTPUT)
# ===============================

import os
import joblib
import numpy as np
import pandas as pd

# ---------------------------
# Paths
# ---------------------------
BASE_DIR = r"C:\Users\uthay\Desktop\CyberThreats_FinancialLoss_Prediction_ML_Final_Project"
DEPLOY_PATH = os.path.join(BASE_DIR, "deployment")
PROCESSED_PATH = os.path.join(BASE_DIR, "data", "processed")
MODEL_FILE = os.path.join(DEPLOY_PATH, "CatBoost_step4_binary_prob_tuned.joblib")
PROD_MODEL_FILE = os.path.join(DEPLOY_PATH, "production_model.joblib")
X_TEST_FILE = os.path.join(PROCESSED_PATH, "X_test_binary.npy")
Y_TEST_FILE = os.path.join(PROCESSED_PATH, "y_test_binary.npy")
PROB_CUTOFF_FILE = os.path.join(PROCESSED_PATH, "step4_prob_cutoffs.csv")

# ---------------------------
# Load model
# ---------------------------
best_model = joblib.load(MODEL_FILE)
print(f"✅ Best model loaded: {MODEL_FILE}")

# ---------------------------
# Load preprocessed test data
# ---------------------------
X_test = np.load(X_TEST_FILE)
y_test = np.load(Y_TEST_FILE).astype(int)

# ---------------------------
# Load tuned probability cutoff for CatBoost
# ---------------------------
cutoffs_df = pd.read_csv(PROB_CUTOFF_FILE)
prob_cutoff = float(cutoffs_df.loc[cutoffs_df["Model"]=="CatBoost","Cutoff"].values[0])

# ---------------------------
# Predict with probability cutoff
# ---------------------------
if hasattr(best_model, "predict_proba"):
    test_probs = best_model.predict_proba(X_test)[:, 1]
else:
    test_probs = best_model.decision_function(X_test)
    test_probs = (test_probs - test_probs.min()) / (test_probs.max() - test_probs.min() + 1e-9)

y_pred = (test_probs > prob_cutoff).astype(int)

# ---------------------------
# Save production model
# ---------------------------
joblib.dump(best_model, PROD_MODEL_FILE)

# ---------------------------
# Only show key output
# ---------------------------
print(f"🏆 Best model based on Test Macro-F1: CatBoost")
print(f"🚀 Production model saved at: {PROD_MODEL_FILE}")


✅ Best model loaded: C:\Users\uthay\Desktop\CyberThreats_FinancialLoss_Prediction_ML_Final_Project\deployment\CatBoost_step4_binary_prob_tuned.joblib
🏆 Best model based on Test Macro-F1: CatBoost
🚀 Production model saved at: C:\Users\uthay\Desktop\CyberThreats_FinancialLoss_Prediction_ML_Final_Project\deployment\production_model.joblib
